
## **Evaluación II** ##

### *Evaluación de contenidos 2 Módulo 2* ###

In [36]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
import mysql.connector

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" 
from geopy.geocoders import Nominatim


Como en evaluaciones anteriores dispondréis de 2 tardes (aproximadamente 8 horas para la resolución de todos los ejercicios propuestos).
Estos ejercicios se tendrán que defender en el día de la evaluación de forma individual con vuestra tutora docente en una simulación de entrevista técnica donde tendréis de 20 minutos para resolver o explicar los ejercicios propuestos entregados.

En esta evaluación nos enfrentamos a un desafío emocionante de trabajar en un proyecto real para una empresa que realiza un estudio de universidades por el mundo. El proyecto tiene como objetivo identificar todas las universidades ubicadas en tres países específicos: Estados Unidos, Canadá y Argentina.

Para llevar a cabo esta tarea, utilizaremos la API de "Universities Hipolabs", una fuente confiable y completa de información sobre las universidades en todo el mundo. Con la ayuda de esta API, podemos acceder a una gran cantidad de datos relevantes, incluyendo el nombre de la universidad, la ciudad donde esta ubicada, el nombre de la institución y otra información importante que nos permitirá llevar a cabo un análisis detallado.

Es importante tener en cuenta que este proyecto requerirá un conocimiento profundo de herramientas y técnicas de análisis de datos, así como habilidades en programación y manejo de APIs. También es importante tener una comprensión sólida de la estructura y organización de los datos, ya que esto nos permitirá hacer preguntas importantes y obtener respuestas significativas a partir de los datos.

En resumen, esta prueba técnica ofrece una excelente oportunidad para demostrar habilidades y conocimientos en análisis de datos y programación, mientras se trabaja en un proyecto real y relevante para una empresa. Al finalizar del proyecto, esperamos obtener información valiosa que ayudará a la empresa a tomar decisiones más informadas sobre las universidades en los tres países objetivo.

1. Utilizando la API extraed toda la información que podáis de ella. La url para hacer las llamadas es:


2. Una vez tengáis todos los datos de la API, deberéis realizar una serie de procesos de limpieza, estos incluyen:
Cambiad los nombres de las columnas para homogeneizarlas, tenemos columnas que tienen - y otras _. Unifícalo para que todo vaya con _.
La columna de domains nos da una información similar a la de web_pages. Eliminad la columna domains.




3. Si exploramos la columna de web_pages, nos daremos cuenta que hay universidades, como por ejemplo la Universidad de "Cégep de Saint-Jérôme" de Canadá que en su columna de web_pages tiene más de un valor dentro de la lista. Esto es poco práctico y puede llegar a no tener sentido. el objetivo de este ejericio es que usando el método explode de pandas separéis cada elemento de la lista en una fila nueva.  tenéis la documentación de este método.

4. Una vez hayáis realizado el explode, chequead si tenéis duplicados basándonos unicamente en el nombre de la universidad, en caso de que si, eliminandlos.
   


   
5. Si exploramos la columna de state_province veremos que hay universidades cuyo valor para esta columna es None. Cread una función para reemplazar los None por nulos de numpy.
   


6. Después del último cambio, os habréis dado cuenta que tenemos muchos valores nulos dentro de la columna de state_province, por lo que nuestro jefe nos pide que reemplacemos esos nulos por "Unknow". No nos piden ningún método especifico, asi que podremos usar el método que queramos.
   


### Definimos la lista de variables ###

In [37]:
country_list = ["Argentina", "Canada", "United States"]

### Creamos la función de llamada ###

In [38]:

def API_call(country_list):
    
    """ Esta función accede a la API y extrae información sobre los países asignados. 
        Args: una lista de paises (se facilita en el constructor de la clase)
        Return: Un dataframe que concatena la información de todos los paises asignados """
    
    df_unified = pd.DataFrame()
    
    for c in country_list:

        url = f'http://universities.hipolabs.com/search?country={c}'

        response = requests.get(url)
        status_code = response.status_code
        status_reason = response.reason
        if  status_code == 200:
            print('The request was succeful. Status code:', status_code,'. Reason:',status_reason)
        elif  status_code == 402:
            print('Non-authorized user. Status code:',  status_code,'. Reason:',status_reason)
        elif  status_code == 404:
            print('Information not found. Status code:',  status_code,'. Reason:',status_reason)
        else:
            print('Unexpected error. Status code:',  status_code,'. Reason:',status_reason)

        df = pd.DataFrame.from_dict(pd.json_normalize(response.json()))
        
        df_unified = pd.concat([df_unified, df], axis = 0)
        
    return df_unified
    








### Llamamos la función ###

In [39]:
df_unified = API_call(country_list)

The request was succeful. Status code: 200 . Reason: OK
The request was succeful. Status code: 200 . Reason: OK
The request was succeful. Status code: 200 . Reason: OK


### Creamos la función de limpieza ###


In [40]:
def clean(df_unified):
    
    """ Esta función limpia el dataframe, cambia el formato de algunas columnas, elimina columnas duplicadas y innecesarias y cambia nulos por strings
        Args: dataframe
        Return: El dataframe limpio y transformado"""
    
    dict_provinces = {"NV":"Nevada", "TX":"Texas", "IN": "Indianapolis", "CA":"California", "VA": "Virgina", 
                  "New York, NY": "New York", "MI":"Michigan", "GA": "Georgia", "ND":"North Dakota", "Ciudad Autónoma de Buenos Aires": "Buenos Aires"}
    
    new_columns = {col: col.replace("-", "_") for col in df_unified.columns}
    df_unified.rename(columns = new_columns, inplace = True)
   
    df_unified.drop(['domains'], axis=1, inplace=True)
    
    df_unified = df_unified.explode('web_pages')
    
    df_unified = df_unified.drop_duplicates(subset= 'name')
    
    df_unified['state_province'] = df_unified['state_province'].fillna(value=pd.np.nan)
    
    df_unified['state_province'].replace(dict_provinces, inplace= True)
    
    df_unified['state_province'].fillna(value= 'unknown', inplace= True)
    
    return df_unified
    
    
    
    

### Llamamos la función ###

In [41]:
df_unified = clean(df_unified)

/var/folders/qs/lczv8kqx5w59ygqwfs1w0vlr0000gn/T/ipykernel_4110/3897419100.py:19: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df_unified['state_province'] = df_unified['state_province'].fillna(value=pd.np.nan)


In [42]:
df_unified 

,state_province,name,country,web_pages,alpha_two_code
0,Buenos Aires,Universidad Atlantida Argentina,Argentina,http://www.atlantida.edu.ar/,AR
1,Buenos Aires,Universidad Austral Buenos Aires,Argentina,http://www.austral.edu.ar/,AR
2,Buenos Aires,"Universidad CAECE, Buenos Aires",Argentina,http://www.caece.edu.ar/,AR
3,Buenos Aires,Instituto Universitario CEMA,Argentina,http://www.cema.edu.ar/,AR
4,Buenos Aires,Instituto de Enseñanza Superior del Ejército,Argentina,http://www.iese.edu.ar/,AR
...,...,...,...,...,...
2276,unknown,Virginia University of Lynchburg,United States,https://www.vul.edu/,US
2277,unknown,Voorhees University,United States,https://www.voorhees.edu/,US
2278,unknown,West Virginia State University,United States,https://www.wvstateu.edu/,US
2279,unknown,Wiley College,United States,https://www.wileyc.edu/,US


   
7. Ahora nuestros jefes nos piden que saquemos las coordenadas de las provincias donde están ubicadas las universidades. Para eso nos piden que usemos la librería de geopy que aprendimos el día del repaso,  la documentación. Para desarrollar este ejercicio deberéis:
Sacar los valores únicos de la columna state_province.

Algunos de los valores que tenemos están con siglas, y deberéis reemplazarlos por lo siguiente:

NV: reemplazalo por Nevada
TX: reemplazalo por Texas
IN: reemplazalo por Indianapolis
CA: reemplazalo por California
VA: reemplazalo por Virginia
NY: reemplazalo por New York
MI: reemplazalo por Michigan
GA: reemplazalo por Georgia
ND: reemplazalo por North Dakota

Otros valores que tenemos más formateados son y que deberemos reemplazar:
New York, NY. Deberéis reemplazarlo por "New York".

'Buenos Aires', 'Ciudad Autónoma de Buenos Aires'. En este caso deberéis poner en ambos casos "Buenos Aires"

Una vez realizados los pasos anteriores, crea una lista con los valores únicos de las provincias de las universidades.

Usando la API de geopy, extraed la latitud y la longitud de cada una de las provincias y almacenad los resultados en un dataframe.
Una vez que tengáis los datos del ejercicio anterior en un dataframe, unidlo con el de las universidades que hemos sacado de la API.


### Creamos la función de coordenadas ###

In [45]:
def coords(df_unified):
    """Esta función permite obtener las coordenadas de latitud y longitud de la localización asignada y mergea el resulatado a un dataframe asignado.
    Args: un parámetro, que es el dataframe.
    Return: el dataframe total unido"""
    
    provinces = df_unified['state_province'].unique()
    list_lat= []
    list_lon = []

    for prov in provinces:

        geolocator = Nominatim(user_agent="Laura")
        loc = geolocator.geocode(prov)
        list_lat.append(loc.latitude)
        list_lon.append(loc.longitude)
        

    dict_coords = {"state_province": provinces, "latitud": list_lat, "longitud": list_lon}
    df_coords = pd.DataFrame(dict_coords)
    df_total =  df_unified.merge(df_coords, on = "state_province")
   
    
    return df_total

In [47]:
coords(df_unified)

,state_province,name,country,web_pages,alpha_two_code,latitud,longitud
0,Buenos Aires,Universidad Atlantida Argentina,Argentina,http://www.atlantida.edu.ar/,AR,-34.607568,-58.437089
1,Buenos Aires,Universidad Austral Buenos Aires,Argentina,http://www.austral.edu.ar/,AR,-34.607568,-58.437089
2,Buenos Aires,"Universidad CAECE, Buenos Aires",Argentina,http://www.caece.edu.ar/,AR,-34.607568,-58.437089
3,Buenos Aires,Instituto Universitario CEMA,Argentina,http://www.cema.edu.ar/,AR,-34.607568,-58.437089
4,Buenos Aires,Instituto de Enseñanza Superior del Ejército,Argentina,http://www.iese.edu.ar/,AR,-34.607568,-58.437089
...,...,...,...,...,...,...,...
2502,Georgia,Morehouse,United States,https://morehouse.edu/,US,32.329381,-83.113737
2503,New York,CUNY Lehman College,United States,https://www.lehman.cuny.edu/,US,40.712728,-74.006015
2504,New York,CUNY John Jay College of Criminal Justice,United States,https://www.jjay.cuny.edu/,US,40.712728,-74.006015
2505,New York,CUNY Hunter College,United States,https://hunter.cuny.edu/,US,40.712728,-74.006015


In [54]:
df_unified.to_csv("../DA-promoD-Mod2-sprint2-LauraMadrid/df_unified.csv")


8. Crea una BBDD en mysql que contenga las siguientes tablas:
   
- Tabla países: donde encontraremos las siguientes columnas:
  
  - idestado: primary key, integer, autoincremental
  - nombre_pais: varchar
  - nombre_provincia: varchar
  - latitud: decimal
  - longitud: decimal
  -----------------

- Tabla universidades: donde encontraremos las siguientes columnas:
  
  - iduniversidades: primary key, integer, autoincremental
  - nombre_universidad: varchar
  - pagina_web: varchar
  - paises_idestado: foreing key

### Conexión con el servidor de SQL y Creación BBDD

In [1]:

def create_bbdd(nombre_bbdd):
    
    """Esta función crea la base de datos para la posterior creación de tablas en la misma. 
    Args: nombre de la base de datos"""

    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="AlumnaAdalab" 
    )
    print("Conexión realizada con éxito")
    
    mycursor = mydb.cursor()

    try:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
            

In [127]:
mydb = create_bbdd("universidades")

Conexión realizada con éxito
CMySQLCursor: CREATE DATABASE IF NOT EXISTS universida..


### Creación de tablas

In [3]:
def create_insert_table(nombre_bbdd, contraseña, query):
        
    """Esta función crea las tablas a seren insertadas en la base de datos.
    Args: nombre de la base de datos, contraseña y la query de creación."""
    cnx = mysql.connector.connect(user='root', password=f"{contraseña}",
                                     host='127.0.0.1', database=f"{nombre_bbdd}")
    
    mycursor = cnx.cursor()
    
  
    try: 
        mycursor.execute(query)
        cnx.commit() 
   
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
        

In [135]:
Tabla_países = '''
CREATE TABLE IF NOT EXISTS `universidades`.`Tabla_países` (
  `idestado` INT NOT NULL AUTO_INCREMENT, 
  `nombre_pais` VARCHAR(45) NULL,
  `nombre_provincia` VARCHAR(45) NULL,
  `latitud` DECIMAL NULL,
  `longitud` DECIMAL NULL,
  PRIMARY KEY (`idestado`))
ENGINE = InnoDB;'''


Tabla_universidades = '''
CREATE TABLE IF NOT EXISTS `universidades`.`Tabla_universidades` (
  `iduniversidades` INT NOT NULL AUTO_INCREMENT,
  `nombre_universidad` VARCHAR(100) NULL,
  `pagina_web` VARCHAR(100) NULL,
  `paises_idestado` VARCHAR(45) NULL,
  `Tabla_países_idestado` INT NOT NULL,
  PRIMARY KEY (`iduniversidades`),
  INDEX `fk_Tabla_universidades_Tabla_países_idx` (`Tabla_países_idestado` ASC) VISIBLE,
  CONSTRAINT `fk_Tabla_universidades_Tabla_países`
    FOREIGN KEY (`Tabla_países_idestado`)
    REFERENCES `universidades`.`Tabla_países` (`idestado`)
    ON DELETE CASCADE
    ON UPDATE CASCADE)
ENGINE = InnoDB;'''




In [130]:
create_insert_table("universidades", 'AlumnaAdalab',Tabla_países)

In [136]:
create_insert_table("universidades", 'AlumnaAdalab',Tabla_universidades)

10. BONUS

- Introduce los datos en la BBDD de SQL.
- Crea una clase con todo el código generado en esta evaluación.
